# Policy Gradient Reinforcement Learning

### **Team Member:** 108024507 張文騰 / 108024512 吳紹丞 / 108024519 劉怡禎 / 109062659 蘇瑞揚

Below, we divide our training code into 3 parts which are PPO, A3C, and PG. And we have tried framed-based RL based on PPO.

# **Content**
#### 0. **Game Statistics**
        - EDA result
#### 1. **PPO** 
        - State-Based
#### 2. **A3C**
        - State-Based
#### 3. **PG**
        - State-Based
#### 4. **Conclusion**

# Set up Environment

In [ ]:
import sys
sys.version

'3.7.9 (default, Aug 31 2020, 12:42:55) \n[GCC 7.3.0]'

In [ ]:
import tensorflow as tf
import numpy as np
import os

print(os.getcwd())

/home/ingmember03/DL2020/DL2020_07/comp4


In [ ]:
# limit the uasge of memory
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=500)])
    except RuntimeError as e:
        print(e)

In [ ]:
os.environ["SDL_VIDEODRIVER"] = "dummy"  # this line make pop-out window not appear
from ple.games.flappybird import FlappyBird
from ple import PLE

game = FlappyBird()
env = PLE(game, fps=30, display_screen=False)  # environment interface to game
env.reset_game()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


# 0. Game Statistics

#### Game EDA

In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage

env.reset_game()
T = 0
rewards = []
frames = []
states = []

In [ ]:
# to pass first pipe
# 0 -> 38 times -> 7
# 1 -> 22 times -> go 

rewards.append(env.act(env.getActionSet()[0]))
states.append(TA_state())
print(game.getGameState())
frames.append(env.getScreenRGB())
plt.imshow(ndimage.rotate(frames[T], 270))
print("\nT: {} REWARD: {}".format(T, rewards[T]))

T += 1

rewards.append(env.act(env.getActionSet()[1]))
states.append(TA_state())
print(game.getGameState())
frames.append(env.getScreenRGB())
plt.imshow(ndimage.rotate(frames[T], 270))
print("\nT: {} REWARD: {}".format(T, rewards[T]))

T += 1

1. T = 61 -> REWARD = -5
2. T = 97 -> REWARD = -5
3. T = 18 -> Earliest Dead  
4. if player_y > 390 -> die
5. if player_y < -5 -> die
6. next_pipe_dist_to_player is the distance between player and the exit point of pipe, which means if one hits the pipe directly at first, there are still 61 units away from the exit point of pipe.

**Conclusion:**
 - Reward function should weigh up while passing thru the pipe, the inequality form is 61 <= next_pipe_dist_to_player <= 0

# 1. PPO - state-based

### Define Make Movie Function

In [ ]:
import moviepy.editor as mpy

def make_anim(images, fps=60, true_image=False):
    duration = len(images) / fps

    def make_frame(t):
        try:
            x = images[int(len(images) / duration * t)]
        except:
            x = images[-1]

        if true_image:
            return x.astype(np.uint8)
        else:
            return ((x + 1) / 2 * 255).astype(np.uint8)

    clip = mpy.VideoClip(make_frame, duration=duration)
    clip.fps = fps
    return clip

### Define Actor

In [ ]:
Actor = tf.keras.Sequential()
# Actor.add(tf.keras.layers.Dense(32, input_dim = 8, activation='relu',kernel_initializer = 'random_uniform', bias_initializer = 'random_uniform'))
Actor.add(tf.keras.layers.Dense(32, input_dim = 8, kernel_initializer = 'random_uniform', bias_initializer = 'random_uniform'))
Actor.add(tf.keras.layers.LeakyReLU(alpha=0.3))
Actor.add(tf.keras.layers.Dense(32, kernel_initializer = 'random_uniform', bias_initializer = 'random_uniform'))
Actor.add(tf.keras.layers.LeakyReLU(alpha=0.3))
Actor.add(tf.keras.layers.Dense(64,  kernel_initializer = 'random_uniform', bias_initializer = 'random_uniform'))
Actor.add(tf.keras.layers.LeakyReLU(alpha=0.3))
Actor.add(tf.keras.layers.Dense(2,kernel_initializer = 'random_uniform', bias_initializer = 'random_uniform'))
Actor.add(tf.keras.layers.Softmax())
Actor.build()
Actor_opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
print(Actor.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                288       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                2112      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                

### TA_state

In [ ]:
import copy

def TA_state():
    state = copy.deepcopy(game.getGameState())
    
    state['next_next_pipe_bottom_y'] -= state['player_y']
    state['next_next_pipe_top_y'] -= state['player_y']
    state['next_pipe_bottom_y'] -= state['player_y']
    state['next_pipe_top_y'] -= state['player_y']
    relative_state = list(state.values())


    # # return the state in tensor type, with batch dimension
    # relative_state = tf.convert_to_tensor(relative_state, dtype=tf.float32)
    # relative_state = tf.expand_dims(relative_state, axis=0)
    
    return relative_state

### Define Forward Advantage Function (Not Efficient)

In [ ]:
def Advantage_func(rewards, time, gamma):
    dis_rewards = []
    for t in range(1,time+1,1):
        t_prime = t
        dis_reward = 0
        count = t - 1
        for i in range(count,len(rewards),1):
            dis_reward += rewards[i] * (gamma ** (t_prime-t))
            t_prime += 1
        dis_rewards.append(dis_reward)
    
    # naive baseline
    # baseline = np.mean(dis_rewards)
    # Advantage = dis_rewards-baseline
    Advantage = dis_rewards
    return (Advantage)

### Define Backward Advantage Function (Efficient)

In [ ]:
def Advantage_func_fromback(rewards, time, gamma):
    dis_rewards = []
    dis_reward = 0
    count = 0

    for t in range(time,0,-1):
        dis_reward = dis_reward * gamma + rewards[t-1]
            
        dis_rewards.append(dis_reward)
        count += 1
    
    # naive baseline
    # baseline = np.mean(dis_rewards)
    # Advantage = dis_rewards-baseline
    Advantage = list(reversed(dis_rewards))
    return (Advantage)

#### Check the result of two advantage functions (should be the same)

In [ ]:
r = np.random.normal(size = 1000, loc = 0, scale = 1)
print(r[:5])
print(Advantage_func(r, len(r), 0.9)[:5]) # as size goes large, it'll take too much time
print(Advantage_func_fromback(r, len(r), 0.9)[:5])

[-0.06909741  1.58449479  0.89380297  0.76317422  1.11642288]
[3.0468995689243488, 3.4622188605162063, 2.0863600803735247, 1.3250634612221566, 0.6243213826610516]
[3.0468995689243443, 3.4622188605162036, 2.086360080373523, 1.3250634612221557, 0.624321382661053]


### Define Loss Function (-Objective Function) with For Loop (Not Efficient)

In [ ]:
def J_func(probs, old_probs, adv, epsilon):
    J = []
    for up, op, a in zip(probs, old_probs, adv):
        p_ratio = up/op
        s1 = (p_ratio * a)
        s2 =  (tf.clip_by_value(p_ratio, 1-epsilon, 1+epsilon) * a)
        J.append(-tf.math.minimum(s1,s2))

    return(J)

In [ ]:
r = [40,30,20,10]
adv = Advantage_func_fromback(r, len(r), 0.9)
print(adv)
probs = [0.8,0.7,0.6,0.5]
old_probs = [0.7,0.6,0.6,0.7]

J_func(probs, old_probs, adv, 0.05)

[90.49000000000001, 56.1, 29.0, 10.0]


[<tf.Tensor: shape=(), dtype=float32, numpy=-95.014496>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-58.904995>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-29.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-7.142857>]

### Define Loss Function (-Objective Function) with TF API (Efficient)

In [ ]:
def J_func_tf(probs, old_probs, adv, epsilon):

    p_ratio = tf.divide(probs,old_probs)
    s1 = tf.multiply(p_ratio,tf.cast(adv, dtype = tf.float32))
    s2 = tf.multiply(tf.clip_by_value(p_ratio, 1-epsilon, 1+epsilon), tf.cast(adv, dtype = tf.float32))
    J = -tf.math.minimum(s1,s2)

    return(J)

In [ ]:
r = [40,30,20,10]
adv = Advantage_func_fromback(r, len(r), 0.9)
print(adv)
probs = tf.constant([0.8,0.7,0.6,0.5])
old_probs = tf.constant([0.7,0.6,0.6,0.7])
adv = tf.constant(adv)

J_func_tf(probs, old_probs, adv, 0.05)

[90.49000000000001, 56.1, 29.0, 10.0]


<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-95.014496 , -58.904995 , -29.       ,  -7.1428576], dtype=float32)>

### Define Learning Function

In [ ]:
EPSILON = 0.1

def train_step(states, actions, adv, probs, ep):

    with tf.GradientTape() as tape:
        prs = []
        pr = Agent(states)

        # print(pr)
        for idx, a in enumerate(actions):
            prs.append(pr[idx][a])

        prs = tf.stack(prs, axis = 0)
        probs = tf.constant(probs)
        adv = tf.stack(adv, axis = 0)

        EXP_J = J_func_tf(prs, probs, adv, EPSILON)
        actor_loss = (tf.math.reduce_mean(EXP_J))
        # print("P: ", prs,'\n')
        # print("loss: ", actor_loss,'\n')
        
    grads = tape.gradient(actor_loss, Agent.trainable_variables)
    Actor_opt.apply_gradients(zip(grads, Agent.trainable_variables))

    return actor_loss

#### Define Reward Function - Add some rewards and penalties thru the road to pipe
 - When 141 >= next_distance_to_pipe > 61, if bird is higher or lower than the top or bottom of the pipe, penalty = -0.5; if between, reward = 1.5
 - When 61 >=  next_distance_to_pipe > 0, if bird is higher or lower than the top or bottom of the pipe, penalty = 0; if between, reward = 2.5
 - Note that the original reward and penalty in game are 1 and -5 when bird flies through the pipe or die

We add additional rewards criteria (first 2 bullet points) onto the original rewards in PPO.

In [ ]:
REWARD_DIST = 20

def Reward_func(states, rewards):

    Rewards_adj = []

    for i in range(1, len(states), 1):
        
        next_state = states[i][0]
        
        if next_state[2] > 61 and next_state[2] <= 61 + 4 * REWARD_DIST:
            if next_state[0] <= next_state[3]+next_state[0]:
                re = -0.5
            elif next_state[0] >= next_state[4]+next_state[0]:
                re = -0.5
            else:
                re = 1.5 # randomly assign a value
        # the minimum of next_pipe_distance = 1
        elif next_state[2] <= 61 and next_state[2] > 0:
            if next_state[0] <= next_state[3]+next_state[0]:
                re = 0
            elif next_state[0] >= next_state[4]+next_state[0]:
                re = 0
            else:
                re = 2.5 # randomly assign a value
        else:
            re = 0

        re = tf.dtypes.cast(re, tf.float32)
        Rewards_adj.append(re)

    Rewards_all = tf.constant(rewards) + Rewards_adj 

    return Rewards_all

#### Start Training

In [ ]:
# model_path = "./model_rewardsum442/"
# Agent = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
# tf.random.set_seed(1)

Agent = Actor

NUM_EPISODE = 50000
GAMMA = 0.95
EXPLORE_RATIO_STAGE1 = 0.85
EXPLORE_RATIO_STAGE2 = 0.9
EXPLORE_LIMIT_STAGE1 = 5000 
EXPLORE_LIMIT_STAGE2 = 8000
EXPLORE_LIMIT_CEILING = 10000
START_PPO_UPDATE = 60 
EPOCHS = 10

best = 200
Cum_reward = []
Ts = []

iter_num = 0
for episode in range(0, NUM_EPISODE + 1, 1):

    # Reset the environment
    env.reset_game()

    frames = [env.getScreenRGB()]
    
    cum_reward = 0
  
    # all_aloss = []
    # all_closs = []
    rewards = []
    states = []
    actions = []
    old_probs = []
    # values = []

    # feed current state and select an action
    state = tf.constant(np.array(TA_state()).reshape(1,8))
    states.append(state)

    T = 0
    print("EPISODE: {}".format(episode))
    while not env.game_over():

        # feed current state and select an action
        Stochastic = Agent(state)[0].numpy()

        # Exploration
        if episode < EXPLORE_LIMIT_STAGE1:
            if Stochastic[0] > EXPLORE_RATIO_STAGE1:
                Stochastic[0] = EXPLORE_RATIO_STAGE1
                Stochastic[1] = 1 - EXPLORE_RATIO_STAGE1
            elif Stochastic[0] < 1-EXPLORE_RATIO_STAGE1:
                Stochastic[0] = 1 - EXPLORE_RATIO_STAGE1
                Stochastic[1] = EXPLORE_RATIO_STAGE1

        if episode >= EXPLORE_LIMIT_STAGE2 & episode < EXPLORE_LIMIT_CEILING:
            if Stochastic[0] > EXPLORE_RATIO_STAGE2:
                Stochastic[0] = EXPLORE_RATIO_STAGE2
                Stochastic[1] = 1 - EXPLORE_RATIO_STAGE2
            elif Stochastic[0] < 1-EXPLORE_RATIO_STAGE2:
                Stochastic[0] = 1 - EXPLORE_RATIO_STAGE2
                Stochastic[1] = EXPLORE_RATIO_STAGE2
        
        action = np.random.choice(2 ,p =  Stochastic)
        prob = Stochastic[action]
        # value = Agent.critic(state).numpy()

        # execute the action and get reward
        reward = env.act(env.getActionSet()[action])

        frames.append(env.getScreenRGB())
        
        # collect trajectory
        actions.append(action)
        rewards.append(reward) 
        old_probs.append(prob)
        # values.append(value)
        
        state = np.array(TA_state()).reshape(1,8)
        states.append(state)
        
        if T>500 and T%100 == 0:
            print("T_IN_TRAJECTORY: {}".format(T))
        T += 1

    if T>500:
        print("MAX_T_BEFORE_PPO_STAGE2: {}".format(T))
    
    # value = Agent.critic(state).numpy()
    # values.append(value)

    # print(states)

    Rewards = Reward_func(states, rewards)
    cum_reward = np.sum(Rewards)
    states = tf.constant(np.array(states[:-1]).reshape(len(states[:-1]),8) )   # [[[],[]]]
    actions = np.array(actions, dtype=np.int32)
    Cum_reward.append(np.round(cum_reward,3))
    Ts.append(T)
    

    print(Rewards)
    # CALCULATE ADVANTAGE BASED ON THE NEW REWARDS
    adv = Advantage_func_fromback(Rewards, len(Rewards), GAMMA)

    # print("EPISODE: ",episode,'\n')
    # print("STATES: ",states,'\n')
    # print("PROBS: ",old_probs,'\n')
    # print("ACTIONS: ",actions,'\n')
    # print("REWARDS: ",rewards,'\n')
    # print("NEW REWARDS: ",Rewards,'\n')
    # print("ADVANTAGE: ", adv,'\n')
    # print("CUM REWARDS: ", cum_reward,'\n')

    if T <= START_PPO_UPDATE:
        actor_loss = train_step(states, actions, adv, old_probs, episode)
        print("[{}] epochs: {}".format(episode, 0))
        print("LOSS: {}".format(actor_loss))
    else: 
        for epochs in range(EPOCHS):
            actor_loss = train_step(states, actions, adv, old_probs, episode)
            print("[{}] epochs: {}".format(episode, epochs))
            print("LOSS: {}".format(actor_loss))
        START_PPO_UPDATE = T

    print(
        "time_live: {}\ncumulated reward: {}\navg_time_live: {}\navg_cum_reward: {}\nmax_time_live: {}\nmax_cum_reward: {}\n".
        format(T, np.round(cum_reward,3), np.round(np.mean(Ts)), np.round(np.mean(Cum_reward),3), np.max(Ts), np.round(np.max(Cum_reward),3)))

    if (T>best):
        print('\n')
        # Agent.save('test{}.h5'.format(T))
        tf.keras.models.save_model(Agent, filepath='./model_rewardsum{}/'.format(T))
        print('\n')
        clip = make_anim(frames, fps=60, true_image=True).rotate(-90)
        clip.write_videofile("/home/ingmember03/DL2020/DL2020_07/comp4/PPO_rewardsum_demo-{}.webm".format(T), fps=60)
#         display(clip.ipython_display(fps=60, autoplay=1, loop=1, maxduration=120))
        best = T

# pipe1 -> 62 - 78
# pipe2 -> 98 - 114
# pipe3 -> 134 - 150

EPISODE: 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  -0.5
 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5 -0.5
 -0.5 -0.5 -0.5 -0.5 -5.5], shape=(61,), dtype=float32)
[0] epochs: 0
LOSS: 4.6023359298706055
[0] epochs: 1
LOSS: 4.57857608795166
[0] epochs: 2
LOSS: 4.560789108276367
[0] epochs: 3
LOSS: 4.556968688964

# 2. A3C - state-based

### Parameters
Here we define parameters used in A3C. Also, we change the original rewards in the game.

In [ ]:
args = {
  'gamma' : 0.9,
  'update_interval':300,
  'actor_lr':0.001,
  'critic_lr':0.001,
  'entropy_beta':0.05,
  'reward_no_die':0.01,
  'reward_die':-5,
  'reward_through':1.5
}
CUR_EPISODE = 0

### Define Actor and Critic model

In [ ]:
class Actor:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(args['actor_lr'])
        self.entropy_beta = args['entropy_beta']

    def create_model(self):
        return tf.keras.Sequential([
            Input((self.state_dim,)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_dim, activation='softmax')
        ])

    def compute_loss(self, actions, logits, advantages):
        ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True)
        entropy_loss = tf.keras.losses.CategoricalCrossentropy(
            from_logits=True)
        actions = tf.cast(actions, tf.int32)
        policy_loss = ce_loss(
            actions, logits, sample_weight=tf.stop_gradient(advantages))
        entropy = entropy_loss(logits, logits)
        return policy_loss - self.entropy_beta * entropy

    def train(self, states, actions, advantages):
        with tf.GradientTape() as tape:
            logits = self.model(states, training=True)
            loss = self.compute_loss(
                actions, logits, advantages)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss

class Critic:
    def __init__(self, state_dim):
        self.state_dim = state_dim
        self.model = self.create_model()
        self.opt = tf.keras.optimizers.Adam(args['critic_lr'])

    def create_model(self):
        return tf.keras.Sequential([
            Input((self.state_dim,)),
            Dense(256, activation='relu'),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(1, activation='linear')
        ])

    def compute_loss(self, v_pred, td_targets):
        mse = tf.keras.losses.MeanSquaredError()
        return mse(td_targets, v_pred)

    def train(self, states, td_targets):
        with tf.GradientTape() as tape:
            v_pred = self.model(states, training=True)
            assert v_pred.shape == td_targets.shape
            loss = self.compute_loss(v_pred, tf.stop_gradient(td_targets))
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss

### Define Global Agent
In A3C, we have to define a global model, which collect games played from all threads in CPU and then update the global model parametors.

In [ ]:
class Agent:
    def __init__(self, env_name):
        env = make_new()
        self.env_name = env_name
        self.state_dim = TA_state(env).shape[1]
        self.action_dim = len(env.getActionSet())

        self.global_actor = Actor(self.state_dim, self.action_dim)
        self.global_critic = Critic(self.state_dim)
        self.num_workers = cpu_count()
        #self.num_workers = 1
    def train(self, max_episodes=1000000):
        workers = []
        for i in range(self.num_workers):
            env = make_new()
            workers.append(WorkerAgent(
                env, self.global_actor, self.global_critic, max_episodes))

        for worker in workers:
            worker.start()

        for worker in workers:
            worker.join()

### Define Local Agent
This agent is used to play in threads. Note that we update the global parametors once after collecting 300 games.

In [ ]:
class WorkerAgent(Thread):
    def __init__(self, env, global_actor, global_critic, max_episodes):
        Thread.__init__(self)
        self.lock = Lock()
        self.env = env
        self.state_dim = TA_state(self.env).shape[1]
        self.action_dim = len(self.env.getActionSet())

        self.max_episodes = max_episodes
        self.global_actor = global_actor
        self.global_critic = global_critic
        self.actor = Actor(self.state_dim, self.action_dim)
        self.critic = Critic(self.state_dim)

        self.actor.model.set_weights(self.global_actor.model.get_weights())
        self.critic.model.set_weights(self.global_critic.model.get_weights())

    def n_step_td_target(self, rewards, next_v_value, done):
        td_targets = np.zeros_like(rewards)
        cumulative = 0
        if not done:
            cumulative = next_v_value

        for k in reversed(range(0, len(rewards))):
            cumulative = args['gamma'] * cumulative + rewards[k]
            td_targets[k] = cumulative
        return td_targets

    def advatnage(self, td_targets, baselines):
        return td_targets - baselines

    def list_to_batch(self, list):
        batch = list[0]
        for elem in list[1:]:
            batch = np.append(batch, elem, axis=0)
        return batch

    def train(self):
        global CUR_EPISODE

        while self.max_episodes >= CUR_EPISODE:
            state_batch = []
            action_batch = []
            reward_batch = []
            episode_reward, done = 0, False
            self.env.reset_game()

            state = TA_state(self.env)
            total_loss =0
            while not done:
                probs = self.actor.model.predict(state)
                action = np.random.choice(self.action_dim, p=probs[0])

                reward = reward_trans(self.env.act(self.env.getActionSet()[action])) 
                
                next_state = TA_state(self.env)
                done = self.env.game_over()

                action = np.reshape(action, [1, 1])
                reward = np.reshape(reward, [1, 1])

                state_batch.append(state)
                action_batch.append(action)
                reward_batch.append(reward)

                if len(state_batch) >= args['update_interval'] or done:
                    states = self.list_to_batch(state_batch)
                    actions = self.list_to_batch(action_batch)
                    rewards = self.list_to_batch(reward_batch)

                    next_v_value = self.critic.model.predict(next_state)
                    td_targets = self.n_step_td_target(
                        rewards, next_v_value, done)
                    advantages = td_targets - self.critic.model.predict(states)
                    
                    with self.lock:
                        actor_loss = self.global_actor.train(
                            states, actions, advantages)
                        critic_loss = self.global_critic.train(
                            states, td_targets)

                        self.actor.model.set_weights(
                            self.global_actor.model.get_weights())
                        self.critic.model.set_weights(
                            self.global_critic.model.get_weights())
                    
                    total_loss+=actor_loss
                    total_loss+=critic_loss

                    state_batch = []
                    action_batch = []
                    reward_batch = []
                    td_target_batch = []
                    advatnage_batch = []

                episode_reward += reward[0][0]
                state = next_state
            if CUR_EPISODE % 100 == 0:    
                print('EP{} EpisodeReward={} TotalLoss={}\n'.format(CUR_EPISODE, episode_reward,total_loss))
            wandb.log({'Reward': episode_reward,'Total Loss':total_loss})
            CUR_EPISODE += 1

### Start training

In [ ]:
agent = Agent("flappy_bird_A3C")
agent.train(100000)

# 3. PG - state-based

### Training

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
from IPython.display import Image, display
import moviepy.editor as mpy
import tensorflow as tf
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
import copy
from ple.games.flappybird import FlappyBird
from ple import PLE

multiple_return_values = False
gpu_number = 0
seed = 2021

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

game = FlappyBird()
env = PLE(game, fps=30, display_screen=False, rng=seed)  # game environment interface
env.reset_game()

def TA_state():
    state = copy.deepcopy(game.getGameState())
    
    state['next_next_pipe_bottom_y'] -= state['player_y']
    state['next_next_pipe_top_y'] -= state['player_y']
    state['next_pipe_bottom_y'] -= state['player_y']
    state['next_pipe_top_y'] -= state['player_y']
    relative_state = list(state.values())


    # return the state in tensor type, with batch dimension
    relative_state = tf.convert_to_tensor(relative_state, dtype=tf.float32)
    relative_state = tf.expand_dims(relative_state, axis=0)
    
    return relative_state

def MY_reward(n,p):

    a = n['next_pipe_bottom_y']
    b = n['next_pipe_top_y']
    
    re_n = (a+b)/2
    re_n -= n['player_y']
    
    a = p['next_pipe_bottom_y']
    b = p['next_pipe_top_y']
    
    re_p = (a+b)/2
    re_p -= p['player_y']

    
    re_n = -(np.absolute(re_n))
    re_p = -(np.absolute(re_p))

    return (re_n - re_p)/16
  
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32, input_dim = 8, activation='relu', kernel_initializer='random_normal'))
model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer='random_normal'))
model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer='random_normal'))
model.add(tf.keras.layers.Dense(2, activation = "softmax"))
model.build()
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
print(model.summary())

def discount_rewards(r, gamma = 0.5):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

class GradUpdate:
    def __init__(self, model):
        self.Buffer = model.trainable_variables
        self.zero()
        
    def zero(self):
        for ix, grad in enumerate(self.Buffer):
            self.Buffer[ix] = grad * 0
            
    def update(self, ep_memory):
        for grads, r in ep_memory:
            for ix, grad in enumerate(grads):
                self.Buffer[ix] += grad * r
                
def get_action(model, s):
    s = s.reshape([1,4])
    logits = model(s)
    a_dist = logits.numpy()
    # Choose random action with p = action dist
    a = np.random.choice(a_dist[0],p=a_dist[0])
    a = np.argmax(a_dist == a)    
    return logits, a

episodes = 2000
scores = []
update_every = 5
gradBuffer = GradUpdate(model)  

h = 0
for e in range(episodes):
    env.reset_game()
    
    frames = [env.getScreenRGB()]
    
    ep_memory = []
    ep_score = 0
    done = False 
    t = 0
    s_n = game.getGameState()
    
    while not env.game_over(): 
        with tf.GradientTape() as tape:
            #forward pass
            state = TA_state()
            logits = model(state)
            a_dist = logits.numpy()
            #print(a_dist)
            # Choose random action with p = action dist
            action = np.random.choice(a_dist[0],p=a_dist[0])
            action = np.argmax(a_dist == action)                    
            loss = compute_loss([action], logits)
        # make the choosen action 

        reward = env.act(env.getActionSet()[action])
        frames.append(env.getScreenRGB())

        ep_score +=reward
        
        s_p = s_n
        s_n = game.getGameState()
        reward += MY_reward(s_n, s_p)

        grads = tape.gradient(loss, model.trainable_variables)
        ep_memory.append([grads,reward])
        scores.append(ep_score)
        t+=1
    
    if(t>h):
        model.save('test{}.h5'.format(t))
        h = t
    # Discound the rewards 
    ep_memory = np.array(ep_memory)
    ep_memory[:,1] = discount_rewards(ep_memory[:,1])    
    
    gradBuffer.update(ep_memory)        
    if e % update_every == 0:
        optimizer.apply_gradients(zip(gradBuffer.Buffer, model.trainable_variables))
        gradBuffer.zero()

    if e % 100 == 0:
        print("Episode  {}  Score  {}".format(e, np.mean(scores[-100:])))
    print(t)

# 4. Conclusion

- After trying above 3 models, we find out that although original policy gradients quite simple, its performance is still good comparing to others.
- While trying PPO (off-policy), we find that it's not necessary to update more than 1 time in every trajectory, we can just focus on longer trajectories and update many times during training. In addition, during the early epsiodes, it is also not necessary to launch several updates per trajectory, since it doesn't take much time to collect trajectory in the early stage.
- We includes state information to define two different reward functions:
  1. based on the bird height comparing to the middle point of pipe, therefore, we can create a continuous reward function.
  2. based on the distance of the bird to pipe, for example, if bird is 20 time steps away from the pipe, if it flies over or lower than the pipe's top or bottom, we penalize it, o.w., we reward.
- For a well-defined reward function, we suggest that the baseline in advantage function can be removed since rewards are in both positive and negative already.
- While building model, we find out that leaky relu activation function is better than relu.